## Purpose

This script will scan the CKAN API for the Minnesota Geospatial Commons and return the metadata for all items as two CSV files:

1. the GeoBTAA Metadata Application Profile.
2. secondary CSV file for all links


## Import modules

In [2]:
# Standard libraries
import csv
import urllib.request
import json
import time
import os
import re
import ast
import decimal
import ssl
import sys

# Third-party libraries
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Declare paths and defaults

In [3]:
# auto-generate the current time in 'YYYYMM' format
action_date = time.strftime('%Y%m%d')

# # Specify the path to the JSON file
output_filename = "ckan_metadata.json"  # Update with the correct path

# Specify the CKAN portal URL you want to harvest from
portalURL = "https://gisdata.mn.gov/"

# Construct the API URL for package search
packageURL = portalURL + 'api/3/action/package_search'

# Specify the path for the CSV file
csv_file_path = action_date + "_05a-01.csv"  # Update with the desired path

DEFAULT_STATE = "Minnesota"

counties_in_minnesota = [
    'Aitkin', 'Anoka', 'Becker', 'Beltrami', 'Benton', 'Big Stone',
    'Blue Earth', 'Brown', 'Carlton', 'Carver', 'Cass', 'Chippewa',
    'Chisago', 'Clay', 'Clearwater', 'Cook', 'Cottonwood', 'Crow Wing',
    'Dakota', 'Dodge', 'Douglas', 'Faribault', 'Fillmore', 'Freeborn',
    'Goodhue', 'Grant', 'Hennepin', 'Houston', 'Hubbard', 'Isanti',
    'Itasca', 'Jackson', 'Kanabec', 'Kandiyohi', 'Kittson', 'Koochiching',
    'Lac qui Parle', 'Lake', 'Lake of the Woods', 'Le Sueur', 'Lincoln',
    'Lyon', 'Mahnomen', 'Marshall', 'Martin', 'McLeod', 'Meeker',
    'Mille Lacs', 'Morrison', 'Mower', 'Murray', 'Nicollet', 'Nobles',
    'Norman', 'Olmsted', 'Otter Tail', 'Pennington', 'Pine', 'Pipestone',
    'Polk', 'Pope', 'Ramsey', 'Red Lake', 'Redwood', 'Renville',
    'Rice', 'Rock', 'Roseau', 'St. Louis', 'Scott', 'Sherburne',
    'Sibley', 'Stearns', 'Steele', 'Stevens', 'Swift', 'Todd',
    'Traverse', 'Wabasha', 'Wadena', 'Waseca', 'Washington', 'Watonwan',
    'Wilkin', 'Winona', 'Wright', 'Yellow Medicine'
]

cities_in_minnesota = [
    'Minneapolis', 'St. Paul', 'Rochester', 'Duluth', 'Bloomington',
    'Brooklyn Park', 'Plymouth', 'Woodbury', 'St. Cloud', 'Eagan',
    'Maple Grove', 'Eden Prairie', 'Coon Rapids', 'Burnsville', 'Blaine',
    'Lakeville', 'Minnetonka', 'Apple Valley', 'Edina', 'St. Louis Park','Twin Cities'
]

## Download the metadata to your desktop

This cell will scan the API and create a JSON file on your desktop. This will take several minutes.

In [ ]:
# Specify the number of items per page
items_per_page = 10

# Initialize variables for pagination
start = 0
total_results = 0

# List to store all metadata
all_metadata = []

# Request metadata in paginated manner
while True:
    # Construct the API request URL with pagination parameters
    api_request_url = f"{packageURL}?start={start}&rows={items_per_page}"
    
    # Request metadata
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(api_request_url, context=context).read()
    response_json = json.loads(response.decode('utf-8'))
    
    # Extract metadata from the response
    metadata = response_json['result']['results']
    all_metadata.extend(metadata)
    
    # Update pagination variables
    start += items_per_page
    total_results = response_json['result']['count']
    
    # Break the loop if we have collected all items
    if start >= total_results:
        break


# Save the metadata to a local JSON file on your desktop
desktop_path = ""  # Replace with your desktop path
output_filename = "ckan_metadata.json"
output_path = os.path.join(desktop_path, output_filename) # More portable way to join paths

with open(output_path, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Metadata for {total_results} items saved to {output_path}")

## Read the JSON into a pandas dataframe

### NOTE

You now have the metadata on your desktop inside the same directory as this Notebook.  If any of the following steps fail, you can come back to reading the JSON into a dataframe. You do not have to download the metadata again

In [4]:
# Read the JSON file into a DataFrame
all_df = pd.read_json(output_filename)

# Specify the columns you want to keep and their new names
columns_to_keep_and_rename = {
    "id": "ID",
    "title": "Alternative Title",
    "notes": "Description",
    "name": "Identifier",
    "extras": "extras",
    "resources": "resources",
    "groups": "groups",
    "tags": "tags"
}

# Select and rename the specified columns
df = all_df[list(columns_to_keep_and_rename.keys())].rename(columns=columns_to_keep_and_rename)

### Flatten the tags and groups arrays to create Keyword and Theme

In [5]:
# copy the values for tags into Keyword

df["Keyword"] = df["tags"].apply(lambda x: "|".join([tag["display_name"] for tag in x]) if isinstance(x, list) else "")

# Some of the Theme values need to be renamed
theme_mapping = {
    "Biota": "Biology",
    "Geoscientific": "Geology",
    "Imagery + Basemaps": "Imagery|Land Cover",
    "Planning + Cadastre": "Property",
    "Utilities + Communication": "Utilities"
}

# Function to flatten the "groups" and apply the theme mapping
def flatten_groups(groups):
    if isinstance(groups, list):
        group_values = [group["display_name"] for group in groups]
        return "|".join([theme_mapping.get(value, value) for value in group_values])
    return ""

# Use the flatten_groups function to create the "Theme" column
df["Theme"] = df["groups"].apply(flatten_groups)

# Drop the "groups" and "tags" columns
df.drop(columns=['groups', 'tags'], inplace=True)

In [6]:
# Optional check: Write the selected DataFrame to a CSV file
# At this point there will be 9 columns 
# ID
# Alternative 
# Title
# Description
# Identifier
# extras - arrays of various fields
# resources - arrays of links
# Keyword
# Theme

# df.to_csv(csv_file_path, index=False)

### Flatten extras into a dictionary and expand into DataFrame

In [7]:
flattened_extras_df = df["extras"].apply(lambda x: {item["key"]: item["value"] for item in x}).apply(pd.Series)

# Specify the columns you want to keep from flattened extras and their new names
extras_to_keep_and_rename = {
    "dsAccessConst": "Rights",
    "dsCurrentRef": "Currentness",
    "dsMetadataUrl": "metadata_html",
    "dsOriginator": "Creator",
    "dsPeriodOfContent": "Temporal Coverage",
    "dsPurpose": "Purpose",
    "spatial": "spatial"
}

# Select, rename, and concatenate the required extras columns
df = pd.concat([df, flattened_extras_df[list(extras_to_keep_and_rename.keys())].rename(columns=extras_to_keep_and_rename)], axis=1)

# Combine the "Purpose", "Currentness", and "Description" columns, using '|' as a delimiter
df['Description'] = df['Description'].fillna('') + '|' + df['Purpose'].fillna('') + '|' + df['Currentness'].fillna('')

# Drop the columns that were moved
df.drop(columns=['extras', 'Purpose', 'Currentness'], inplace=True)

### Convert the WKT coordinates into W,S,E,N for Bounding Box

In [8]:
def get_bounding_box(wkt_str):
    # Default bounding box for Minnesota
    default_bounding_box = "-97.2392,43.4994,-89.4919,49.3845"

    # Check if the value is missing or not a string
    if wkt_str is None or not isinstance(wkt_str, str):
        return default_bounding_box

    try:
        # Parse the string as JSON
        wkt_json = json.loads(wkt_str)

        # Extract the coordinates
        coordinates = wkt_json["coordinates"][0]

        # Calculate the bounding box
        west = min(coord[0] for coord in coordinates)
        south = min(coord[1] for coord in coordinates)
        east = max(coord[0] for coord in coordinates)
        north = max(coord[1] for coord in coordinates)

        # Return the bounding box in the required format
        return f"{west},{south},{east},{north}"

    except (json.JSONDecodeError, KeyError, TypeError):
        return default_bounding_box

# Apply the function to the 'spatial' column and create a new 'Bounding Box' column
df['Bounding Box'] = df['spatial'].apply(get_bounding_box)

# Drop the spatial column
df.drop(columns=['spatial'], inplace=True)

### Get place names based on Creator

In [9]:
def get_spatial_coverage(originator):
    if pd.isnull(originator): # Check for NaN
        return DEFAULT_STATE

    coverage_strings = []  # List to store coverage strings

    # Check for counties
    for county in counties_in_minnesota:
        if county in originator:
            coverage_strings.append(f"{DEFAULT_STATE}--{county} County")

    # Check for specific organizations
    if "Metropolitan Council" in originator or "MetroGIS" in originator:
        coverage_strings += [f"{DEFAULT_STATE}--{county} County" for county in ["Anoka", "Carver", "Dakota", "Hennepin", "Ramsey", "Scott", "Washington"]]
        coverage_strings.append(f"{DEFAULT_STATE}--Twin Cities Metropolitan Area")
    elif "Metropolitan Emergency Services Board" in originator:
        coverage_strings += [f"{DEFAULT_STATE}--{county} County" for county in ["Anoka", "Carver", "Chisago", "Dakota", "Hennepin", "Isanti", "Ramsey", "Scott", "Sherburne", "Washington"]]
        coverage_strings.append(f"{DEFAULT_STATE}--Twin Cities Metropolitan Area")
    
    # If no specific coverage strings were found, return the default state
    if not coverage_strings:
        return DEFAULT_STATE
    
    # Join all the coverage strings with the default state at the end
    return "|".join(coverage_strings) + f"|{DEFAULT_STATE}"

# Apply the function to the 'dsOriginator' column and create a new 'Spatial Coverage' column
df['Spatial Coverage'] = df['Creator'].apply(get_spatial_coverage)

### Reformat Temporal Coverage and get the Date Range

In [10]:
# Convert the "Temporal Coverage" column to a datetime object with the correct format
# Using errors='coerce' to handle incorrect date formats
df['Temporal Coverage'] = pd.to_datetime(df['Temporal Coverage'], errors='coerce', format='%m/%d/%Y')

# Convert the datetime object to the desired string format
df['Temporal Coverage'] = df['Temporal Coverage'].dt.strftime('%Y-%m-%d')

# Handle NaT values by replacing them with a default value or NaN
df['Temporal Coverage'].replace('NaT', pd.NA, inplace=True)

def get_date_range(date):
    if pd.isnull(date):  # Check for NaN
        return None
    year = str(date).split('-')[0]  # Extract the year from the date string
    return f"{year}-{year}"  # Format the date range

# Apply the function to the 'Temporal Coverage' column and create a new 'Date Range' column
df['Date Range'] = df['Temporal Coverage'].apply(get_date_range)


### Format the title

In [11]:
def transform_with_date(alt_title, date):
    # Transform the title with the existing logic
    transformed_title = transform_title(alt_title)

    # Check if date is not missing and is a string
    if isinstance(date, str) and '/' in date:
        # Extract the year from the date string
        year = date.split('/')[-1]

        # Append the year in curly brackets
        transformed_title += f" {{{year}}}"

    return transformed_title


def transform_title(alt_title):
    bracket_content = None
    
    # 1. Look for ", {county} County, Minnesota" and replace with "[Minnesota--{county} County]"
    for county in counties_in_minnesota:
        if f", {county} County, Minnesota" in alt_title:
            bracket_content = f"[Minnesota--{county} County]"
            alt_title = re.sub(f", {county} County, Minnesota", "", alt_title, 1)

    # 2. Look for "{city}, Minnesota" and replace with "[Minnesota--{city}]"
    if not bracket_content:
        for city in cities_in_minnesota:
            if f"{city}, Minnesota" in alt_title:
                bracket_content = f"[Minnesota--{city}]"
                alt_title = re.sub(f"{city}, Minnesota", "", alt_title, 1)

    # 3. Look for variations of just the state of Minnesota
    if not bracket_content:
        variations = [r", Minnesota", r"For Minnesota", r"Of Minnesota", r"In Minnesota"]
        for variation in variations:
            if variation.lower() in alt_title.lower():
                bracket_content = "[Minnesota]"
                alt_title = re.sub(variation, "", alt_title, flags=re.IGNORECASE)

    # 4. For everything else, add ["Minnesota"] to the end
    if not bracket_content:
        bracket_content = "[Minnesota]"

    # Append bracketed content to the end of the title
    alt_title = f"{alt_title.strip()} {bracket_content}"

    return alt_title

df['Title'] = df.apply(lambda row: transform_with_date(row['Alternative Title'], row['Temporal Coverage']), axis=1)

### Clean up Creators

In [12]:
def transform_publisher(publisher):
    # Dictionary mapping of publishers for direct transformation
    publisher_mappings = {
        "Metropolitan Council" : "Metropolitan Council of the Twin Cities Area",
        "U.S. Geological Survey" : "Geological Survey (U.S.)"

    }
    
    # If a direct mapping is found, return the transformed value
    if publisher in publisher_mappings:
        return publisher_mappings[publisher]
    
    # Check for values that start with "Minnesota Department of Natural Resources"
    if publisher.startswith("Minnesota Department of Natural Resources"):
        return "Minnesota. Department of Natural Resources"
    
    # Search for a county name in the publisher string.
    for county in counties_in_minnesota:
        if county + " County" in publisher:
            return f"Minnesota--{county} County"
    else:
        for city in cities_in_minnesota:
            if f"City of {city}" in publisher or city == publisher:
                return f"Minnesota--{city}"
    
    # If no match found, return the original publisher string.
    return publisher

df['Creator'] = df['Creator'].apply(transform_publisher)

### Remove duplicates

In [13]:
#The below code removes records that are already covered by other harvests
values_to_remove = [
    "b903cc92-6740-475b-8a27-4f69bb2436ff",
    "912ae6b8-cf8b-4336-b392-3226e6e9bc86",
    "d58e62bb-cb41-474e-b200-1220759f3f7e",
    "16b4ade6-398a-4aaa-8290-9fabe3a70c63",
    "a11d5da8-a932-4c54-a5d5-9946d0dbfe28",
    "09d1f9dc-0a26-4a2c-bb28-f866e10af95e",
    "6b6a200c-7791-40b2-8063-e348691b6ff8",
    "6073b93f-68c0-4316-8346-59f63beb62e6",
    "bdfa0cd7-84bc-4017-8124-98803f594e4f",
    "e87315d6-a614-45ad-878b-66e8337b21cb",
    "906ae99c-cc6c-486b-bf7c-72c1ef9c046c",
    "573c2362-bc9a-4031-aa60-3b0fee613471"
]

df = df[~df['ID'].isin(values_to_remove)]

creators_to_remove = [
    "Minnesota Geological Survey",
    "Minnesota Geological Survey (MGS)"
]

df = df[~df['Creator'].isin(creators_to_remove)]

In [14]:
# Optional check: Write the selected DataFrame to a CSV file
# at this point there will be 15 columns 
# ID
# Alternative Title
# Description
# Identifier
# resources - arrays of links
# Keyword
# Theme
# Rights
# HTML
# Creator
# Temporal Coverage
# Bounding Box
# Spatial Coverage
# Date Range
# Title

# df.to_csv(csv_file_path, index=False)

## Parse the URLs

### Use the URL types to determine Resource Class

In [15]:
# loop through the resources, assign web service types, and determine the Resource Class
# Add new columns and initialize with None


df['Display Note'] = None
df['Resource Class'] = None

def classify_urls(row):
    for resource in row['resources']:
        resource_type = resource.get('resource_type', '')
        url = resource.get('url', '')
        
        # Classify based on resource_type
        if resource_type == 'ags_mapserver':
            row['arcgis_dynamic_map_layer'] = url
        elif resource_type == 'ags_featureserver':
            row['arcgis_feature_layer'] = url
        elif resource_type == 'ags_imageserver':
            row['arcgis_image_layer'] = url

        # Add a display note for recognized ArcGIS services
        if resource_type in ['ags_mapserver', 'ags_featureserver', 'ags_imageserver']:
            creator = row.get('Creator', 'Unknown Creator')
            row['Display Note'] = (
                f"Tip: This page displays a preview of one layer from the {creator} ArcGIS Rest Service. "
                f"However, if the web service is in a different project or contains multiple layers, it may not display. "
                f"Try opening the layer in ArcGIS Online or click the 'Visit Source' to browse for other layers in the service."
            )
                       
         # Handle the other resource types
        dataset_types = ['fgdb', 'gpkg', 'shp', 'csv', 'xlsx', 'kmz', 'cad', 'aaigrid', 'geojson', 'json']
        imagery_type = 'tif'

        # Check if the "Resource Class" has not been set or it's set to 'Other'
        if pd.isna(row.get('Resource Class')):
            if resource_type in dataset_types:
                row['Resource Class'] = 'Datasets'
        elif resource_type == imagery_type:
            row['Resource Class'] = 'Imagery'
                
    return row


# Apply the function to the DataFrame
df = df.apply(classify_urls, axis=1)


In [16]:
# Assign a Resource Class to the remaining items

# Define a pattern that matches 'image' or 'photo' anywhere in the string, without capturing groups
pattern = r'(?:image|photo)'

# Identify rows where the "Title" column contains the pattern and the "Resource Class" is NaN
mask = df['Title'].str.contains(pattern, case=False, na=False) & pd.isna(df['Resource Class'])

# Set the "Resource Class" for those rows to "Imagery"
df.loc[mask, 'Resource Class'] = 'Imagery'

# Fill missing values in the "Resource Class" column with the string "Other"
df['Resource Class'] = df['Resource Class'].fillna('Other')

### Write the Distributions file

In [17]:
# Mapping from JSON resource_type to reference_type
resource_type_mapping = {
    "ags_mapserver": "arcgis_dynamic_map_layer",
    "ags_featureserver": "arcgis_feature_layer",
    "ags_imageserver": "arcgis_image_map_layer",
    "ags_tiled": "arcgis_tiled_map_layer",
    "preview": "image",
    "metadata": "metadata_html",
    # Add any additional mappings as needed
}

# List of download formats with labels
download_formats = {
    "fgdb": "Geodatabase",
    "gpkg": "GeoPackage",
    "shp": "Shapefile",
    "csv": "CSV",
    "xlsx": "XLSX",
    "tif": "TIF",
    "cad": "CAD",
    "kmz": "KMZ",
    "geojson": "GeoJSON",
    "json": "JSON",
    "aaigrid": "ArcGRID"
}

# Combine resource type and download mappings for reference
valid_resource_types = set(resource_type_mapping.keys()).union(download_formats.keys())
combined_mapping = {**resource_type_mapping, **{key: "download" for key in download_formats}}

# List to collect rows for the new secondary CSV
secondary_rows = []

# Iterate through the original DataFrame to process resources
for index, row in df.iterrows():
    friendlier_id = row['ID']
    resources = row['resources']

    # Iterate through each resource in the "resources" field
    for resource in resources:
        resource_type = resource.get('resource_type', '')
        url = resource.get('url', '')
        name = resource.get('name', 'Unspecified')

        # Keep only explicitly listed resource types
        if resource_type not in valid_resource_types:
            continue

        if pd.notna(url):
            # Determine reference_type and label
            reference_type = combined_mapping.get(resource_type, "other")  # Default to "other" if no match
            label = download_formats.get(resource_type, name if name else resource_type)

            secondary_rows.append({
                "friendlier_id": friendlier_id,
                "reference_type": reference_type,
                "distribution_url": url,
                "label": label
            })

# Add the constructed landing page link
for index, row in df.iterrows():
    friendlier_id = row['ID']
    landing_page_url = f"https://gisdata.mn.gov/dataset/{friendlier_id}"
    secondary_rows.append({
        "friendlier_id": friendlier_id,
        "reference_type": "full_layer_description",
        "distribution_url": landing_page_url,
        "label": "Landing Page"
    })

# Create a DataFrame for the secondary CSV
secondary_links_df = pd.DataFrame(secondary_rows)

# Write the DataFrame to a CSV file with the required column order
secondary_links_df = secondary_links_df[
    ["friendlier_id", "reference_type", "distribution_url", "label"]
]
secondary_links_df.to_csv("05a-01_secondary-links.csv", index=False)

print("Secondary CSV for downloads and links created as '05a-01_secondary-links.csv'")




Secondary CSV for downloads and links created as '05a-01_secondary-links.csv'


### Add default and constructed values

In [18]:
df['Date Accessioned'] = action_date
df['Code'] = "05a-01"
df['Is Part Of'] = "05a-01"
df['Member Of'] = "ba5cc745-21c5-4ae9-954b-72dd8db6815a"
df['Accrual Method'] = "CKAN"
df['Access Rights'] = "Public"
df['Language'] = "eng"
df['Provider'] = "Minnesota Geospatial Commons"
df['Format'] = "Files"

In [19]:
# Define the desired order of columns and drop unused columns
desired_order = [
'Title',
'Alternative Title',
'Description',
'Language',
'Format',
'Display Note',
'Creator',
'Provider',
'Resource Class',
'Theme',
'Temporal Coverage',
'Date Range',
'Spatial Coverage',
'Bounding Box',
'Member Of',
'Is Part Of',
'ID',
'Identifier',
'Rights',
'Access Rights',
'Date Accessioned',
'Code',
'Accrual Method'

# Add more columns as needed in the desired order
]

# Reindex the DataFrame based on the desired order of columns
df = df.reindex(columns=desired_order)


In [20]:
# Apply .str.strip() method to all string columns in the DataFrame and replace newline and tab characters
df = df.apply(lambda x: x.str.replace('\n', ' ').str.replace('\t', ' ').str.replace('<br/>', ' ').str.replace('<br/><br/>', '|').str.strip() if x.dtype == "object" else x)

In [21]:
# Write the selected DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)